# Qcodes example with the NI RFSG signal generator driver

The generic `NI_RFSG` driver used below should be compatible with most NI signal generatros (see the driver documentation for list of devices). For some devices, a device-specific driver is available, which differs from the generic one only in that it has appropriate limits set for the `Parameter`s.

In [1]:
# generic driver
from qcodes_contrib_drivers.drivers.NationalInstruments.RFSG import NI_RFSG

# device-specific driver, used in the same way as the generic one
#from qcodes_contrib_drivers.drivers.NationalInstruments.PXIe_5654 import NI_PXIe_5654

## Initialize the instrument

The `resource` name should be the name you have set up in NI MAX.

In [2]:
pxie5654 = NI_RFSG("PXIe5654", resource="MW_source")

Connected to: National Instruments NI PXIe-5654 (10 GHz) (serial:03176E2E, firmware:Firmware: 2015-03-01 21:42, Bootloader: 8) in 1.68s


## Generate a signal

Set a power level (in dBm) and frequency, and start signal generation.

In [3]:
pxie5654.power_level(-2)
pxie5654.frequency(5e9)
pxie5654.output_enabled(True)

The signal should be visible in your oscilloscope now.

In [4]:
pxie5654.output_enabled(False)

## Generate a pulsed signal

In [5]:
pxie5654.output_enabled(True)
print(f"using clock source '{pxie5654.clock_source()}'")
pxie5654.pulse_mod_enabled(True)

using clock source 'onboard'


Now signal output is on only when there is TTL high level voltage present on the "pulse in" connector at the instrument's front panel.

In [6]:
pxie5654.pulse_mod_enabled(False)

Now signal output is on regardless of voltage level present on "pulse in" connector

## Analog (amplitude / phase / frequency) modulation

In this example, we show amplitude modulation (AM). You should connect a modulation signal (for example from an other RF source or an AWG) to the AM IN port on the front of the device for this to work.

With analog modulation disabled, the unmodulated carrier signal output by the MW source is $A_c \cos(2\pi f t)$. If you connect a modulation signal $m(t)$ to the AM IN port, the output signal will nominally be

$$
(m(t) + A_c)\cos(2\pi f t).
$$

Note that connecting a 0V signal to the modulation port produces a continuous tone at the base amplitude $A_c$. In order to suppress the carrier wave, you need to send a negative voltage to the AM IN port.

In [3]:
pxie5654.analog_mod_type()

'none'

In [4]:
# turn on signal generation
pxie5654.output_enabled(True)

Enable amplitude modulation. After running the cell below, the amplitude of the signal should be modulated by the signal sent to AM IN.

In [21]:
pxie5654.analog_mod_type("AM")

The AM sensitivity can be adjusted with the `amplitude_mod_sensitivity` parameter.

In [23]:
pxie5654.amplitude_mod_sensitivity()

100.0

In [24]:
pxie5654.amplitude_mod_sensitivity(50)

In [25]:
pxie5654.amplitude_mod_sensitivity(100)

Turn off analog modulation and revert to normal continuous signal generation with a constant amplitude.

In [75]:
pxie5654.analog_mod_type("none")

## Stop RF generation

In [27]:
pxie5654.output_enabled(False)

## Take a snapshot

In [8]:
pxie5654.print_readable_snapshot(update=True)

PXIe5654:
	parameter        value
--------------------------------------------------------------------------------
IDN               :	{'vendor': 'National Instruments', 'model': 'NI PXIe-5654...
clock_source      :	onboard 
frequency         :	5e+09 (Hz)
output_enabled    :	False 
power_level       :	-2 (dBm)
pulse_mod_enabled :	False 


## Close the instrument

In [9]:
pxie5654.close()